# Problem

    By replacing the 1st digit of the 2-digit number *3, it turns out that six of the nine possible values: 13, 23, 43, 53, 73, and 83, are all prime.

    By replacing the 3rd and 4th digits of 56**3 with the same digit, this 5-digit number is the first example having seven primes among the ten generated numbers, yielding the family: 56003, 56113, 56333, 56443, 56663, 56773, and 56993.
    Consequently 56003, being the first member of this family, is the smallest prime with this property.

    Find the smallest prime which, by replacing part of the number (not necessarily adjacent digits) with the same digit, is part of an eight prime value family.

# Solution

    First at all, we are probabry going to deal with huge numbers, therefore we need a fast algorithms to check if number is prime. Eratosthenes sieve has o(n^2) time complexity and becomes really slow above 1000. 
    I spent some time searching for a good algorithm checking if number is prime and found the Miller_Rabin_test. The idea of this algorithm is quite hard for understanding (at least for me), so I've just copied the code as a function and I am going to use it main function.

In [4]:
from itertools import product

In [136]:
def try_composite(a, d, n, s):
    if pow(a, d, n) == 1:
        return False
    for i in range(s):
        if pow(a, 2 ** i * d, n) == n - 1:
            return False
    return True 

def Miller_Rabin_test(n, precision_for_huge_n=16):
    if n in known_primes or n in (0, 1):
        return True
    if any((n % p) == 0 for p in known_primes):
        return False
    d, s = n - 1, 0
    while not d % 2:
        d, s = d >> 1, s + 1
    if n < 1373653:
        return not any(try_composite(a, d, n, s) for a in (2, 3))
    if n < 25326001:
        return not any(try_composite(a, d, n, s) for a in (2, 3, 5))
    if n < 118670087467:
        if n == 3215031751:
            return False
        return not any(try_composite(a, d, n, s) for a in (2, 3, 5, 7))
    if n < 2152302898747:
        return not any(try_composite(a, d, n, s) for a in (2, 3, 5, 7, 11))
    if n < 3474749660383:
        return not any(try_composite(a, d, n, s) for a in (2, 3, 5, 7, 11, 13))
    if n < 341550071728321:
        return not any(try_composite(a, d, n, s) for a in (2, 3, 5, 7, 11, 13, 17))
    if n < 3825123056546413051:
        return not any(try_composite(a, d, n, s) for a in (2, 3, 5, 7, 11, 13, 17, 19, 23))
    if n < pow(2, 64):
        return not any(try_composite(a, d, n, s) for a in (2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37))
    # otherwise
    return not any(try_composite(a, d, n, s) for a in known_primes[:precision_for_huge_n])

known_primes = [2, 3]
known_primes += [x for x in range(5, 1000, 2) if Miller_Rabin_test(x)]

    Let's define a main function, which will be doing all possible digit replacements for following primes.
    This function will also return a prime if n results of digit replacements are primes.
    
    But before that I am going to create an inside function prime_replacements() returning a single set of replacements.
    In this way we'll make our code more "readable".

In [138]:
def prime_replacements(pattern, num):
    
    repls = []
    
    for i in range(0, 10):
        num_str = str(num)
        
        for j in range(len(pattern)):
            # Check if we are not going to change the first digit to 0.
            # '2009' -> '0009' that converts into 9.
            if i == 0 and j == 0 and pattern[j] == True:
                break
            
            elif pattern[j] == True:
                num_str = num_str[:j] + str(i) + num_str[j+1:]
                
            else:
                continue
        
        number = int(num_str)
        if Miller_Rabin_test(number):
            repls.append(number)
            prime_repl[int(num_str)] = True
            
    return set(repls)

In [144]:
def main():
    
    global prime_repl
    prime_repl = {}
    num = 10
    
    # Generate all possible patterns of replacements with different n for n-digit numbers.
    patterns = {}
    for n in range(2, 12):
        patterns[n] = [x for x in product([True, False], repeat=n-1)][:-1] 
        # We don't need the last pattern as it fully consists of False and means no replacements.
    
    while True:
        num += 1
        
        if num in prime_repl.keys():
            continue
            
        elif Miller_Rabin_test(num):
            
            p = patterns[len(str(num))]
            for pattern in p:
                # Check if all digits in this pattern are the same.
                # For ex. 1009 -> (1019, 2029, 3039, ...) - we change the 1st and 3rd digits.
                # So for the firt elemet in set it appears that we change only one digit.
                digits = {}
                for k in range(len(pattern)):
                    if pattern[k] == True:
                        digits[str(num)[k]] = True
                if len(digits) > 1:
                    continue
                    
                else:    
                    primes = prime_replacements(pattern, num)
                    
                    if len(primes) == 8:
                        return num, primes
        
        else:
            continue

    Importantly, instead of list I use dictionary (prime_repl = {}) to storage the primes, which we don't need to check again in following iterations.
    It seems that list is an optimal solution for adding following primes in list and checking if our number is in it.
    On the other hand, in dictionary we need to write some value for key, which is unnecessary.
    However, checking if value in dictionary is immediate, when the same operation in list takes O(n) time.

# Answer

In [145]:
main()

(424343, {121313, 222323, 323333, 424343, 525353, 626363, 828383, 929393})

    The answer is the lowest value in a set.
 
    Time complexity of algorithm is 10 * 2^(digits in number) * n
    Where 10 - number of digits, 2^(digits in number) - number of patterns for replacements.